Note: Defining Logic for EV Usage and Load (Don't need NREL loads) 
July 2023
Rachel Sheinberg

Step 1: Import Data from Current Interface Type (Excel Spreadsheet)

In [24]:
import os
import pandas as pd
import numpy as np

In [80]:
file_path = r'C:/Users/Rachel/OneDrive - UCLA IT Services/Documents/DWP_Electrification_Cal/electric_calculator.xlsm'
# Read Dataframe from Excel file
inputs = pd.read_excel(file_path, sheet_name='Python')

Calculate Pre-Transition Fuel Cost

In [81]:
current_car = inputs.iloc[13,1]
vmt = inputs.iloc[15,1] if inputs.iloc[15,1] != 0 else 10000
current_mpg = inputs.iloc[14,1]
gas_price = inputs.iloc[10,1]
if current_car == 'None':
    start_gas_cost = 0
elif current_car == 'Gasoline Car/Sedan' or current_car == 'Gasoline Light Truck/Van/SUV':
    ## cost before EV 
    gas_consumed = vmt/current_mpg
    start_gas_cost = gas_price * gas_consumed
else: 
    start_gas_cost = 0 
    ## assuming electric cost is already baked in via load 
start_daily_gas_cost = start_gas_cost/365

In [ ]:
Calculate Post-Transition Electricity Cost 

In [82]:
ev_mpge = 93 if current_car == "Gasoline Light Truck/Van/SUV" else 93
transition = inputs.iloc[12,1]
if transition == 'None':
    upgrade_gas_cost = start_gas_cost
    e_trans_consumed = 0 
    e_trans_consumed_daily = 0 
elif transition == 'Solar PV':
    upgrade_gas_cost = start_gas_cost
    e_trans_consumed = 0
    e_trans_consumed_daily = 0 
else:
    upgrade_gas_cost = 0
    ge_consumed = vmt/ev_mpge
    e_trans_consumed = ge_consumed * 33.7 ## assuming 1 gallon equivalent ~ 33.7 kWh
    e_trans_consumed_daily = e_trans_consumed/365
upgrade_daily_gas_cost = upgrade_gas_cost/365

In [88]:
ev_charging = inputs.iloc[16,1]
### residential charging 
if ev_charging == "Residential/Overnight":
    charge_hours = e_trans_consumed_daily/7.6 #assuming charge rate is 7.6 kW 
    trans_charge_load = e_trans_consumed_daily/charge_hours ## this will be useful for TOU pricing, not using for now
    e_trans_cost_daily = 0 
    ### non-residential charging
elif ev_charging == 'Public or Workplace/Daytime Level 2': 
    e_trans_cost_daily = e_trans_consumed_daily * 0.29 ## assuming $0.29 per kwh
    e_trans_consumed_daily = 0 
elif ev_charging == 'Public or Workplace/Daytime DCFC': 
    e_trans_cost_daily = e_trans_consumed_daily * 0.42 ## assuming $0.42 per kwh
    e_trans_consumed_daily = 0 
else: 
    e_trans_cost_daily = 0 
    e_trans_consumed_daily = 0 
## making an array to add on to monthly sums 
# Define the number of days in each month
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
## start electric cost will always be zero -- assumed to be included in the load
## will just need to add upgrade electric load, upgrade electric cost, start and upgrade gas cost
start_gas_cost = np.array(days_in_month) * start_daily_gas_cost
upgrade_gas_cost = np.array(days_in_month) * upgrade_daily_gas_cost
upgrade_res_ev_load = np.array(days_in_month) * e_trans_consumed_daily
upgrade_res_ev_cost = np.array(days_in_month) * e_trans_cost_daily
print('Monthly EV electricity usage after upgrade (kWh):')
print(upgrade_res_ev_load)   
print('Monthly EV external electricity cost after upgrade ($):') 
print(upgrade_res_ev_cost)
print('Monthly gasoline cost before upgrade ($):')
print(start_gas_cost)
print('Monthly gasoline cost after upgrade ($):')
print(upgrade_gas_cost)

Monthly EV electricity usage after upgrade (kWh):
[307.76255708 277.97908381 307.76255708 297.83473266 307.76255708
 297.83473266 307.76255708 307.76255708 297.83473266 307.76255708
 297.83473266 307.76255708]
Monthly EV external electricity cost after upgrade ($):
[0 0 0 0 0 0 0 0 0 0 0 0]
Monthly gasoline cost before upgrade ($):
[163.06849315 147.28767123 163.06849315 157.80821918 163.06849315
 157.80821918 163.06849315 163.06849315 157.80821918 163.06849315
 157.80821918 163.06849315]
Monthly gasoline cost after upgrade ($):
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
plot(upgrade_res_ev_c)